In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_astradb

In [ ]:
!pip install "unstructured[pptx]"

In [ ]:
!pip install "unstructured[pdf]"

In [ ]:
!mkdir docs

mkdir: cannot create directory ‘docs’: File exists


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
loader=DirectoryLoader("/content/docs")

In [ ]:
splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=64)

In [ ]:
docs=loader.load_and_split(splitter)

In [ ]:
len(docs)

253

In [ ]:
import os
from langchain_core.documents import Document
from langchain_community.document_loaders import AstraDBLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
!pip install transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = "all-mpnet-base-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: list) -> list:
        return self.model.encode(texts).tolist()

    def embed_query(self, text: str) -> list:
        return self.model.encode(text).tolist()

# Now using with LangChain
embeddings = SentenceTransformerEmbeddings()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
ASTRA_DB_API_ENDPOINT="Your_db_endpoint"
ASTRA_DB_APPLICATION_TOKEN="you_astra_db_token"
ASTRA_DB_KEYSPACE="default_keyspace"

In [ ]:
from langchain_astradb import AstraDBVectorStore
vstor=AstraDBVectorStore(
    embedding=embeddings,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    collection_name='vector_store_collection'
)

In [ ]:
# Embed and store the documents in the AstraDB vector store
vstor.add_documents(docs)

['43851a39fb9d4d0889169a6cfd42b491',
 '8e48863f77ad4f778ef54bb0e38c22f1',
 '0b3746985ceb45b9b0ac4f08ca9e53c5',
 '22ad6155768a4ac1a22bc9838dde27b9',
 'ad93f92cc616483092382b7186999ca9',
 '29392258b3d7444abbb8eaf2feda5db0',
 'b8fa9dd8c5ea4cae8b6b56f42c55a663',
 'cadfa16d5002464e9b88a1ce98a1bfb2',
 '67ca7ea13b5d4b65930497b0f61f098e',
 '8a56598b79a84263ace92cf87b723331',
 '6b289ed0e4ce48cd8ab7f2d243d4fb8e',
 'f3fa528e39dd4353b76e848dba893eab',
 'd80127c33ad944ea8e3c898cda9b540c',
 '95390822cafa4eb98839075f782546a6',
 '0ef3e1a93bb7419988d12888474b2dc7',
 'ecd3c75b52b5442c89d2deb441e80ad7',
 '1fa02d4757d7449caf23c0143a8a177e',
 'c0c2cfe488444b1b82cd12c0d32cc518',
 '82e985a585444cacb49dc8a74210057f',
 '3052d77a07f94e4db1d1e60fc90869b3',
 '949fdec9bb4745349f039c06fd062220',
 '14aadd7ed61647579497215165de90dc',
 '1b1484cc0d3145acaa0f004554864e6b',
 '85dfa4b699424587acf0772ca1fce769',
 'aa960cb7e1394621b82f31217b5400eb',
 '841362bd5e7242ba95bfbaee0ad48a90',
 '01714a524b63421eaa2470404dcfc916',
 

In [ ]:
prompt_template="""
You are AI philosopher drawing insights from the roadmap of "rag","langchain","springboot","spring frame work"
Craft thougtful answers based on this roadmap, mixing and matching existing paths.
Your responses should be simple english and strictly related to the provided context.
if then context is not provided or not relevant to question simply say i dont know the answer.
Roadmap context:
{context}
Question:{question}
Your answer:
"""

In [ ]:
prompt_template=ChatPromptTemplate.from_template(prompt_template)

In [ ]:
retriever=vstor.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [ ]:
retriever

VectorStoreRetriever(tags=['AstraDBVectorStore', 'SentenceTransformerEmbeddings'], vectorstore=<langchain_astradb.vectorstores.AstraDBVectorStore object at 0x7ac4edcf9040>, search_type='mmr', search_kwargs={'k': 4})

In [ ]:
!pip install langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.7,
    max_tokens=1024,
    api_key="Your_api_key"
)

In [ ]:
chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    |prompt_template
    |llm
    |StrOutputParser()
)

In [ ]:
chain.invoke("what is spring frameWork")

"Spring Framework is a Java platform that helps build Java applications. It handles the background tasks so you can focus on your app. It's not invasive, meaning you don't have to extend its classes or implement its interfaces. A key part is Dependency Injection, which makes classes not rely on knowing how to create their own dependencies. Spring's IoC Container, represented by interfaces like BeanFactory and ApplicationContext, manages application objects and uses Dependency Injection to achieve this loose coupling. ApplicationContext offers all of BeanFactory's features plus more for enterprise needs."

In [ ]:
print("type 'exit' if you completed asking questions")
while True:
  user_input = input("Enter Your Question: ")
  if user_input.lower() == "exit":
    break
  else:
    print(chain.invoke(user_input))

type 'exit' if you completed asking questions
Enter Your Question: 1+1-1*1
I don't know the answer.
Enter Your Question: What is langchain
LangChain is a framework that helps integrate large language models (LLMs) into applications. It can be used for tasks like document analysis, summarization, chatbots, and code analysis. It was launched in October 2022 and has since received significant funding.
Enter Your Question: Generator class
The `Generator` class is a part of an identifier generation system. It creates unique identifiers for persistent classes. These classes implement the `org.hibernate.id.IdentifierGenerator` interface. Some built-in generator classes include `assigned` (which is the default) and `increment` (which increases the ID value by a specific number). Other types like `SEQUENCE`, `IDENTITY`, and `HILO` are also mentioned for generating identifiers.
Enter Your Question: exit
